In [1]:
from transformers import pipeline

In [2]:
# Load Zero-Shot Classification Model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Define Sentence-Like Labels for Classification
category_labels = [
    "Poor customer service or staff behavior",
    "Quality, taste, or freshness issues with food",
    "Limited food options or menu variety",
    "Ambiance, noise levels, or atmosphere concerns",
    "Concerns about pricing and value",
    "Cleanliness or hygiene issues",
    "Positive review"
]


In [4]:
# Define Thresholds
high_threshold = 0.75  # Threshold to confidently assign complaint categories
positive_threshold = 0.2  # Threshold below which we consider the review as "positive review"


In [ ]:
import pandas as pd
new_dataset = pd.read_csv('combined_dataset.csv')

In [7]:
import pandas as pd

# Function to Classify Reviews in a DataFrame
def classify_reviews_in_dataframe(df, review_column):
    reviews = df[review_column].tolist()  # Extract reviews from the specified column
    # Assuming classifier is defined and working correctly
    results = classifier(reviews, candidate_labels=category_labels, multi_label=True)

    # Prepare lists to hold predicted categories and probabilities
    predicted_categories = []
    probabilities_list = []

    for result in results:
        probabilities = result['scores']
        labels = result['labels']

        # Initialize variables to store predictions
        predicted_category = []
        current_probabilities = []  # Store probabilities for the current review

        # Check for categories with scores greater than 0.75
        for label, score in zip(labels, probabilities):
            if score > 0.75:
                predicted_category.append(label)  # Include high-scoring labels
                current_probabilities.append(score)  # Store their corresponding probabilities

        # If no categories were found with scores > 0.75, check for scores between 0.2 and 0.75
        if not predicted_category:
            max_score = -1
            max_label = None
            for label, score in zip(labels, probabilities):
                if 0.2 <= score < 0.75:
                    # Check for the highest score in this range
                    if score > max_score:
                        max_score = score
                        max_label = label

            # If we found a max_label, add it to the predicted categories
            if max_label is not None:
                predicted_category.append(max_label)
                current_probabilities.append(max_score)

        # If no categories were predicted, assign "positive review"
        if not predicted_category:
            predicted_category = ["positive review"]
            current_probabilities = [1.0]  # Assign full probability for positive review

        # Store results
        predicted_categories.append(predicted_category)
        probabilities_list.append(current_probabilities)  # Store probabilities for the current review

    # Add the results back to the DataFrame
    df['Predicted_Category'] = predicted_categories
    df['Probabilities'] = probabilities_list

    return df

In [11]:
# Assuming `df` is your DataFrame with a column of reviews named 'Cleaned_Review'
sample_reviews_df = new_dataset.sample(5)  # Select 5 random reviews as a sample
classified_sample_df = classify_reviews_in_dataframe(sample_reviews_df, 'Cleaned_Review')

# Display the classified sample reviews with predictions and probabilities
classified_sample_df[['Review', 'Predicted_Category', 'Probabilities']]


,Review,Predicted_Category,Probabilities
72,Four of us went to dine in at noon and the exp...,"[Poor customer service or staff behavior, Conc...","[0.95511394739151, 0.8420560956001282, 0.75955..."
2805,The food options here are quite limited compar...,[Limited food options or menu variety],[0.998332142829895]
3141,Cutlery wasn't clean.\nVery average food.\nBla...,"[Limited food options or menu variety, Quality...","[0.9973154664039612, 0.9859012961387634, 0.972..."
939,Veg starters are good almost same as AB's. Ser...,"[Limited food options or menu variety, Positiv...","[0.9841523170471191, 0.9428579807281494]"
112,Absolutely deplorable service.there were only ...,"[Limited food options or menu variety, Poor cu...","[0.9984270334243774, 0.9976000785827637, 0.989..."


In [ ]:
# Assuming new_dataset is your DataFrame and 'Review' is the column containing reviews
classified_dataset = classify_reviews_in_dataframe(new_dataset, 'Review')


In [ ]:
classified_dataset[['Review', 'Predicted_Category', 'Probabilities']][4100:]

,Review,Predicted_Category,Probabilities
4100,1. The staff here are really friendly and act ...,"[Positive review, Quality, taste, or freshness...","[0.9344078898429871, 0.8381698727607727]"
4101,Have been there on weekend... It was crowded a...,[Positive review],[0.9945999979972839]
4102,One of the best restaurants in Hyderabad hands...,[Positive review],[0.9430651664733887]
4103,Sai is very helpful in terms of knowing what c...,[Positive review],[0.9938072562217712]
4104,Good place to go with frnds and family for Lun...,[Positive review],[0.9979599714279175]
...,...,...,...
4200,Place where food lovers can enjoy their food a...,"[Positive review, Limited food options or menu...","[0.984387218952179, 0.8502863049507141]"
4201,"Good place to visit with friends and family, t...",[Positive review],[0.99602872133255]
4202,"The food is very good, service is also good. B...",[Positive review],[0.9854133129119873]
4203,Nice place to hangout with a small team of upt...,[Positive review],[0.9941960573196411]


In [ ]:
# Define the mapping from existing categories to new labels
category_mapping = {
    "Poor customer service or staff behavior": "Service Issue",
    "Quality, taste, or freshness issues with food": "Food Quality",
    "Limited food options or menu variety": "Food Options",
    "Ambiance, noise levels, or atmosphere concerns": "Atmosphere",
    "Concerns about pricing and value": "Value for Money",
    "Cleanliness or hygiene issues": "Hygiene",
    "Positive review": "Positive Review"
}

In [ ]:

# Map the predicted categories to the new labels
new_dataset['Mapped_Category'] = new_dataset['Predicted_Category'].apply(
    lambda categories: [category_mapping[category] for category in categories if category in category_mapping]
)

In [ ]:
classified_dataset.head()



,Index,Review,Cleaned_Review,Cleaned_Name,Severity,Urgency,Customer_Satisfaction,Review_Days,Response_Days,Predicted_Category,Probabilities,Mapped_Category
0,0,"Stop eating at this place, I have visited bang...",stop eating place visited bangalores nd punes ...,pramod kumar,High,Urgent,No Response,180,-1,"[Quality, taste, or freshness issues with food...","[0.9965850710868835, 0.9946374893188477, 0.844...","[Food Quality, Hygiene, Service Issue]"
1,1,Food 2/5\nService 2/5\nAmbience 2/5 …,food service ambience,abhinav deep,Medium,Non-Urgent,High Satisfaction,365,365,"[Poor customer service or staff behavior, Qual...","[0.8227755427360535, 0.7571902871131897]","[Service Issue, Food Quality]"
2,2,Idiotic varieties for the price they have char...,idiotic varieties price charged varieties boil...,vijay nammi,High,Urgent,No Response,30,-1,"[Quality, taste, or freshness issues with food...","[0.9860756993293762, 0.98301100730896]","[Food Quality, Value for Money]"
3,3,"I am posting this live now, this is one of the...",posting live one worst places dont visit pathe...,surya ajay,High,Urgent,High Satisfaction,365,365,"[Poor customer service or staff behavior, Qual...","[0.992651641368866, 0.9484805464744568]","[Service Issue, Food Quality]"
4,4,"We are pure vegetarians, I ordered veg biryani...",pure vegetarians ordered veg biryani swiggy go...,sai hithesh,Low,Non-Urgent,No Response,180,-1,"[Poor customer service or staff behavior, Conc...","[0.9473494291305542, 0.8612152338027954, 0.833...","[Service Issue, Value for Money, Food Quality]"


In [ ]:
# Create a set of unique categories from the Mapped_Category column
unique_categories = set(category for sublist in classified_dataset['Mapped_Category'] for category in sublist)

# One-hot encode categories
for category in unique_categories:
    classified_dataset[category] = classified_dataset['Mapped_Category'].apply(lambda x: 1 if category in x else 0)

In [ ]:
classified_dataset.head(5)

,Index,Review,Cleaned_Review,Cleaned_Name,Severity,Urgency,Customer_Satisfaction,Review_Days,Response_Days,Predicted_Category,Probabilities,Mapped_Category,Hygiene,Food Quality,Atmosphere,Value for Money,Service Issue,Positive Review,Food Options
0,0,"Stop eating at this place, I have visited bang...",stop eating place visited bangalores nd punes ...,pramod kumar,High,Urgent,No Response,180,-1,"[Quality, taste, or freshness issues with food...","[0.9965850710868835, 0.9946374893188477, 0.844...","[Food Quality, Hygiene, Service Issue]",1,1,0,0,1,0,0
1,1,Food 2/5\nService 2/5\nAmbience 2/5 …,food service ambience,abhinav deep,Medium,Non-Urgent,High Satisfaction,365,365,"[Poor customer service or staff behavior, Qual...","[0.8227755427360535, 0.7571902871131897]","[Service Issue, Food Quality]",0,1,0,0,1,0,0
2,2,Idiotic varieties for the price they have char...,idiotic varieties price charged varieties boil...,vijay nammi,High,Urgent,No Response,30,-1,"[Quality, taste, or freshness issues with food...","[0.9860756993293762, 0.98301100730896]","[Food Quality, Value for Money]",0,1,0,1,0,0,0
3,3,"I am posting this live now, this is one of the...",posting live one worst places dont visit pathe...,surya ajay,High,Urgent,High Satisfaction,365,365,"[Poor customer service or staff behavior, Qual...","[0.992651641368866, 0.9484805464744568]","[Service Issue, Food Quality]",0,1,0,0,1,0,0
4,4,"We are pure vegetarians, I ordered veg biryani...",pure vegetarians ordered veg biryani swiggy go...,sai hithesh,Low,Non-Urgent,No Response,180,-1,"[Poor customer service or staff behavior, Conc...","[0.9473494291305542, 0.8612152338027954, 0.833...","[Service Issue, Value for Money, Food Quality]",0,1,0,1,1,0,0


In [ ]:
classified_dataset.to_csv("Final_Dataset.csv")